This prediction is for  LCLId = MAC000002
* Using ARIMA
* Using MSTL

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import GRU
from neuralforecast.losses.pytorch import DistributionLoss
import pandas as pd
from pprint import pprint

In [ ]:
import sys
sys.path.append("..")
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
db = DuckDBDatabaseHelper("../meters.db")

In [ ]:
train = pd.read_csv("../model_data/train_day.csv")
test = pd.read_csv("../model_data/test_day.csv")

In [ ]:
train.columns

In [ ]:
len(train)

In [ ]:
train.head()

In [ ]:
df = pd.DataFrame()
df["unique_id"] =["Energy"] *len(train)
df["ds"] = train["day"]
df["y"] = train["energy_sum"]
#Add exogenous regressors
# 'temperatureMin', 'temperatureMax', 'uvIndex',
# 'dewPoint', 'Type_Binary'

df['temperatureMin'] = train['temperatureMin']
df['temperatureMax'] = train['temperatureMax']
df['uvIndex'] = train['uvIndex']
df['dewPoint'] = train['dewPoint']
df['Type_Binary'] = train['Type_Binary']


In [ ]:
X_df = pd.DataFrame()
X_df["unique_id"] = ["Energy"] *len(test)
X_df["ds"] = pd.to_datetime(test["day"])
X_df['temperatureMin'] = test['temperatureMin']
X_df['temperatureMax'] = test['temperatureMax']
X_df['uvIndex'] = test['uvIndex']
X_df['dewPoint'] = test['dewPoint']
X_df['Type_Binary'] = test['Type_Binary']

In [ ]:
X_df[:6]

In [ ]:
len(df)

In [ ]:
df = df.dropna()

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
fcst = NeuralForecast(
    models=[GRU(h=31,input_size=-1,
                loss=DistributionLoss(distribution='Normal', 
                                      level=[80, 90]),
                scaler_type='robust',
                encoder_n_layers=2,
                encoder_hidden_size=128,
                context_size=10,
                decoder_hidden_size=128,
                decoder_layers=2,
                max_steps=200
                )
    ],
    freq='D'
)

In [ ]:
df["ds"] = pd.to_datetime(df["ds"])

In [ ]:
fcst.fit(df=df)

In [ ]:
forecast_df = fcst.predict(futr_df=X_df)

In [ ]:
pprint(forecast_df.columns)

### Verify the predictions

In [ ]:
sql_query ="""

SELECT day,energy_sum,
LCLid
FROM  meters m 
WHERE date_part('year', day) = 2014
AND  date_part('month', day) = 1
AND LCLid = 'MAC000002'
"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

In [ ]:
df_LCLid = pd.DataFrame(records)

In [ ]:
df_LCLid.iloc[:,1][:4]

In [ ]:
len(forecast_df["GRU"]),len(df_LCLid.iloc[:,1])

In [ ]:
mae = abs(df_LCLid.iloc[:,1].values-forecast_df["GRU"].values).mean()
print('The MAE is '+str(round(mae,3)))